In [1]:
# import all packages, if error, install using pip install <Package name>
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from nltk.tokenize import word_tokenize
import xml.etree.ElementTree as ET
import os

### Creating dataframe from XML files

In [2]:
# here we will create a list which consists of dictionaries (one for each document)
# dictionary contains tag name as key and its corresponding text as value
rows=[]
for filename in os.listdir(os.getcwd()):
    if 'xml' not in filename:
        continue
    tree = ET.parse(filename)
    root = tree.getroot()
    dic={}
    for child in root:
        key=child.tag
        text=''+child.text.strip()+'\n'
        for fchild in child:
            text=text+fchild.text.strip()+'\n'
        dic[child.tag]=text
    rows.append(dic)

In [6]:
print(rows[0])

{'SubDatabase': 'espacenet\n', 'Title': 'PROCESSES FOR PRODUCING TITANIUM DIOXIDE\n', 'PublicationNumber': 'EP2104645A1\n', 'Inventor': '\nHUTCHENSON KEITH W [US]\nLI SHENG [US]\nCORBIN DAVID RICHARD [US]\nTORARDI CARMINE [US]\nSHIFLETT MARK BRANDON [US]\nZAHER JOSEPH J [US]\nGRIFFIN THOMAS PAUL [US]\nHUTCHENSON, KEITH, W\nLI, SHENG\nCORBIN, DAVID, RICHARD\nTORARDI, CARMINE\nSHIFLETT, MARK, BRANDON\nZAHER, JOSEPH, J\nGRIFFIN, THOMAS, PAUL\n', 'Applicant': '\nDU PONT [US]\nE. I. DU PONT DE NEMOURS AND COMPANY\n', 'RequestedPatent': 'EP2104645\n', 'ApplicationElem': '\nEP20060848328\n', 'ApplicationDate': '2006-12-28\n', 'PriorityElem': '\nWO2006US49556\n2006-12-28\n', 'IPC': '\nC01G23/053\nC01G23/08\nC01G49/02\nC22B34/12\n', 'NCL': '\nB82Y30/00\nC01G23/053\nC01G23/08\nC22B34/12F4\nC22B34/12F6\n', 'Abstract': 'Provided are processes for the production of titanium dioxide from ilmenite. In these processes, ilmenite is digested with aqueous ammonium hydrogen oxalate. Iron from the ilmenite

In [7]:
# column names for creating dataframe 
df_columns=['SubDatabase','Title','PublicationNumber','Inventor',
        'Applicant','RequestedPatent','ApplicationElem',
        'ApplicationDate','PriorityElem','IPC','NCL','Abstract',
        'Claims','Also_published_as']

In [9]:
# each dictionary in rows list is a row for our dataframe which we will create here using pd.DataFrame
df = pd.DataFrame(rows, columns=df_columns)
df.drop_duplicates(inplace=True)
df=df.reset_index().drop(columns=['index'])
print(len(df))
df.head()

1000


,SubDatabase,Title,PublicationNumber,Inventor,Applicant,RequestedPatent,ApplicationElem,ApplicationDate,PriorityElem,IPC,NCL,Abstract,Claims,Also_published_as
0,espacenet\n,PROCESSES FOR PRODUCING TITANIUM DIOXIDE\n,EP2104645A1\n,\nHUTCHENSON KEITH W [US]\nLI SHENG [US]\nCORB...,\nDU PONT [US]\nE. I. DU PONT DE NEMOURS AND C...,EP2104645\n,\nEP20060848328\n,2006-12-28\n,\nWO2006US49556\n2006-12-28\n,\nC01G23/053\nC01G23/08\nC01G49/02\nC22B34/12\n,\nB82Y30/00\nC01G23/053\nC01G23/08\nC22B34/12F...,Provided are processes for the production of t...,\nCLAIMS A process comprising: a) digesting il...,EP2104645B1;WO2008082384A1;MX2009007020A;JP201...
1,espacenet\n,HIGH FREQUENCY DEFLECTION MEASUREMENT OF IR AB...,EP2150785A1\n,\nDAZZI A [FR]\nREADING MICHAEL [GB]\nRUI PRAZ...,\nANASYS INSTR [US]\nDAZZI A [FR]\nREADING MIC...,EP2150785\n,\nEP20080767670\n,2008-05-13\n,\nWO2008US06051\n2008-05-13\nUS20070803421\n20...,\nG01J3/10\nG01J3/30\nG01N21/35\nG01Q30/00\nG0...,\nB82Y35/00\nG01N21/35\nG01Q30/02\nG01Q60/32\n...,An AFM based technique has been demonstrated f...,\nWe claim:\n1. A method of measuring a locali...,EP2150785A4;US2008283755A1;US8001830B2;US20112...
2,espacenet\n,PHOTOVOLTAIC CELL AND METHOD OF MAKING THEREOF\n,EP2115782A1\n,\nKEMPA KRZYSZTOF [US]\nNAUGHTON MICHAEL [US]\...,"\nSOLASTA INC [US]\nSOLASTA, INC\n",EP2115782\n,\nEP20080724892\n,2008-01-29\n,\nWO2008US01113\n2008-01-29\nUS20070887212P\n2...,\nH01L31/04\n,\nB82Y10/00\nB82Y30/00\nH01L31/0352C3\nY02E10/...,A photovoltaic cell includes a first electrode...,"\n1. A photovoltaic cell, comprising:\na first...",US2008178924A1;WO2008094517A1;TW200845404A;KR2...
3,espacenet\n,Reduced striae extreme ultraviolet elements\n,EP2211232A1\n,\nHRDINA KENNETH EDWARD [US]\nMAXON JOHN E [US...,\nCORNING INC [US]\nCORNING INCORPORATED\n,EP2211232\n,\nEP20100157436\n,2003-03-04\n,\nEP20030744207\n2003-03-04\nUS20020362052P\n2...,\nC03B19/14\nC03B20/00\nC03B8/04\nC03C3/06\nC0...,\nB82Y10/00\nC03B19/14C\nC03B19/14G\nC03C3/06\...,Titania-containing silica glass bodies and ext...,\n1. An extreme ultraviolet lithographic eleme...,WO03077038A2;WO03077038A3;USRE40586E1;US200402...
4,espacenet\n,AL(X)GA(1-X)N- CLADDING-FREE NONPOLAR III- NIT...,EP2111634A1\n,\nFEEZELL DANIEL F [US]\nSCHMIDT MATHEW C [US]...,\nUNIV CALIFORNIA [US]\nTHE REGENTS OF THE UNI...,EP2111634\n,\nEP20080725466\n,2008-02-12\n,\nWO2008US01840\n2008-02-12\nUS20070889510P\n2...,\nH01L21/20\nH01L33/32\nH01L33/02\nH01L33/06\n,\nB82Y20/00\nH01L33/32\n,A method for fabricating AlxGa1-xN-cladding-fr...,\nWhat is claimed is:\n1. An optoelectronic de...,US2008191192A1;US8211723B2;WO2008100502A1;WO20...


In [10]:
duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF

,SubDatabase,Title,PublicationNumber,Inventor,Applicant,RequestedPatent,ApplicationElem,ApplicationDate,PriorityElem,IPC,NCL,Abstract,Claims,Also_published_as


In [11]:
# append single column to our dataframe that comrises of all values of a single row merged together
df['documents'] = df[df.columns[1:]].apply(
                                lambda x: ' '.join(x.dropna().astype(str)),axis=1)

In [13]:
documents_df=pd.DataFrame(df['documents'])
documents_df.head()

,documents
0,PROCESSES FOR PRODUCING TITANIUM DIOXIDE\n EP2...
1,HIGH FREQUENCY DEFLECTION MEASUREMENT OF IR AB...
2,PHOTOVOLTAIC CELL AND METHOD OF MAKING THEREOF...
3,Reduced striae extreme ultraviolet elements\n ...
4,AL(X)GA(1-X)N- CLADDING-FREE NONPOLAR III- NIT...


In [14]:
# removing special characters and stop words from the text
stop_words_l=stopwords.words('english')
documents_df['documents_cleaned']=documents_df.documents.apply(
                        lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

### Pairwise similarity

#### Pairwise cosine similarity would just be the dot product of the vectors becasue tf-idf vectors from sklearn are already normalized and L2 norm of these vectors is 1. So denominator of cosine similiarity formula is 1 in this case.

#### similarity is highest, 1 at index 0 becasue they are the same documents

In [29]:
def most_similar(doc_id,similarity_matrix,matrix):
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        dic[ix]=similarity_matrix[doc_id][ix]
        break

### Doc2vec model

In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [17]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(documents_df.documents_cleaned)]

In [18]:
model_d2v = Doc2Vec(vector_size=100,alpha=0.025, min_count=1)
  
model_d2v.build_vocab(tagged_data)

for epoch in range(100):
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.epochs)
    print(epoch)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [19]:
document_embeddings=np.zeros((documents_df.shape[0],100))

for i in range(len(document_embeddings)):
    document_embeddings[i]=model_d2v.docvecs[i]

In [20]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [21]:
documents_df['Cosine Similarity']=0.0
documents_df['Most Similar Document']=0

In [30]:
# here we print the index of document and the corresponding document 
# which is most similar and relevant cosine similarity
for index in documents_df.index:
    dic={}
    most_similar(index,pairwise_similarities,'Cosine Similarity')
    print(index, dic)
    documents_df['Cosine Similarity'][index]=list(dic.items())[0][1]
    documents_df['Most Similar Document'][index]=list(dic.items())[0][0]

0 {20: 0.5798929667955416}
1 {298: 0.5700467154055688}
2 {614: 0.5470019364331375}
3 {544: 0.5584302653188998}
4 {176: 0.576248339278285}
5 {707: 0.6136642336950354}
6 {113: 0.5708214538187809}
7 {627: 0.6458216199970901}
8 {365: 0.506866785003106}
9 {122: 0.4763135927289711}
10 {12: 0.5929891966828895}
11 {58: 0.640073480779944}
12 {10: 0.5929891966828895}
13 {523: 0.48918372282586603}
14 {188: 0.9474359698810108}
15 {601: 0.5587700889010492}
16 {604: 0.5014173798676154}
17 {200: 0.5849410083710911}
18 {548: 0.490661371758009}
19 {45: 0.9954805686680717}
20 {0: 0.5798929667955416}
21 {956: 0.5693063717810969}
22 {611: 0.6642319104652747}
23 {415: 0.5680602331905079}
24 {479: 0.5410056670490695}
25 {951: 0.6768566318217328}
26 {389: 0.6212407892575513}
27 {298: 0.5354395472967448}
28 {845: 0.9162344846211417}
29 {937: 0.4854268054501808}
30 {878: 0.6110290420516384}
31 {140: 0.5031497968401589}
32 {728: 0.5444766938017871}
33 {409: 0.5211498349878156}
34 {994: 0.5284446692733379}
35 {5

<ipython-input-30-7b485987f997>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents_df['Cosine Similarity'][index]=list(dic.items())[0][1]
<ipython-input-30-7b485987f997>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents_df['Most Similar Document'][index]=list(dic.items())[0][0]


549 {773: 0.5707179758688823}
550 {149: 0.5265481151638829}
551 {961: 0.5333223773592628}
552 {282: 0.5034285959595447}
553 {544: 0.5579814206328843}
554 {301: 0.5217184294257164}
555 {884: 0.5729909111588981}
556 {17: 0.5727297117591572}
557 {581: 0.5646321618203811}
558 {407: 0.6895890232693923}
559 {441: 0.5294198323204419}
560 {11: 0.5762284313408017}
561 {724: 0.6446250427178009}
562 {701: 0.4730735252049122}
563 {514: 0.6004920484526615}
564 {897: 0.5142648541553153}
565 {318: 0.7780665996856971}
566 {259: 0.5727557735956699}
567 {267: 0.5857446565450006}
568 {91: 0.7811189622633123}
569 {236: 0.5745786797625565}
570 {542: 0.561625504213593}
571 {636: 0.5401113109339412}
572 {401: 0.6174883630754492}
573 {873: 0.9675092181674545}
574 {277: 0.5811601644872135}
575 {946: 0.8334214010206892}
576 {788: 0.5802428268882955}
577 {398: 0.49935352536979755}
578 {112: 0.5037290836558498}
579 {838: 0.5340154700701165}
580 {881: 0.9918857827952826}
581 {352: 0.8522356818792364}
582 {155: 0.4

966 {640: 0.5788599844985294}
967 {181: 0.6050243356533203}
968 {323: 0.4856610171916927}
969 {146: 0.7308992391941153}
970 {880: 0.5678843968383593}
971 {700: 0.5273289095060407}
972 {39: 0.641308575928079}
973 {698: 0.9000626449062199}
974 {645: 0.9028314341283282}
975 {770: 0.5014034546743471}
976 {429: 0.5586841261678023}
977 {464: 0.6180933009304557}
978 {635: 0.5284091546233682}
979 {567: 0.5726552946040505}
980 {615: 0.5306494443610076}
981 {315: 0.6549838402890319}
982 {561: 0.6022593109610864}
983 {788: 0.5331753560947627}
984 {621: 0.5256608451739517}
985 {195: 0.5428716219375916}
986 {408: 0.5033345628684065}
987 {635: 0.5628746778165472}
988 {164: 0.5216202711058386}
989 {469: 0.5230118303723723}
990 {345: 0.8616547505951075}
991 {631: 0.5515933817989461}
992 {493: 0.4703621480513766}
993 {652: 0.5321609079618848}
994 {34: 0.5284446692733379}
995 {261: 0.6464938121036001}
996 {2: 0.532644652043333}
997 {619: 0.5719285147576333}
998 {406: 0.5573092972187799}
999 {906: 0.6467

In [32]:
# this summarizes the top similar documents in descending order of cosine similarity
documents_df.sort_values(by='Cosine Similarity',ascending=False)

,documents,documents_cleaned,Cosine Similarity,Most Similar Document
19,NONVOLATILE NANOTUBE DIODES AND NONVOLATILE NA...,nonvolatile nanotube diodes nonvolatile nanotu...,0.995481,45
45,"NONVOLATILE RESISTIVE MEMORIES, LATCH CIRCUITS...",nonvolatile resistive memories latch circuits...,0.995481,19
904,MATRIX ASSISTED INK TRANSPORT\n EP2175983A2\n ...,matrix assisted ink transport ep a mirk...,0.994578,850
850,PATTERNING WITH COMPOSITIONS COMPRISING LIPID\...,patterning compositions comprising lipid ep ...,0.994578,904
691,UNIVERSAL MATRIX\n EP2170501A2\n \nMIRKIN CHAD...,universal matrix ep a mirkin chad us ...,0.994556,850
...,...,...,...,...
143,CREATING LAYERS IN THIN- FILM STRUCTURES\n EP2...,creating layers thin film structures ep ...,0.446692,363
114,IMPROVED PARTICLE BEAM GENERATOR\n EP2126955A1...,improved particle beam generator ep a e...,0.445437,754
383,Method and apparatus for synthesis of arrays o...,method apparatus synthesis arrays dna probes e...,0.445022,948
299,Surfaces physically transformable by environme...,surfaces physically transformable environmenta...,0.440367,182


### Most Similar patents are document 19 and document 45 with cosine similarity of 0.993798





In [33]:
df.iloc[[19,45]]

,SubDatabase,Title,PublicationNumber,Inventor,Applicant,RequestedPatent,ApplicationElem,ApplicationDate,PriorityElem,IPC,NCL,Abstract,Claims,Also_published_as,documents
19,espacenet\n,NONVOLATILE NANOTUBE DIODES AND NONVOLATILE NA...,EP2057633A2\n,\nBERTIN CLAUDE L [US]\nRUECKES THOMAS [US]\nH...,"\nNANTERO INC [US]\nNANTERO, INC\n",EP2057633\n,\nEP20070840788\n,2007-08-08\n,\nWO2007US75506\n2007-08-08\nUS20060836343P\n2...,\nG11C13/02\n,\nB82Y10/00\nG11C11/56\nG11C13/00R25V\nG11C13/...,"Under one aspect, a non-volatile nanotube diod...",\nWhat is claimed is:\n1. A non-volatile nanot...,EP2057633A4;WO2008021900A2;WO2008021900A3;WO20...,NONVOLATILE NANOTUBE DIODES AND NONVOLATILE NA...
45,espacenet\n,"NONVOLATILE RESISTIVE MEMORIES, LATCH CIRCUITS...",EP2070088A2\n,\nBERTIN CLAUDE L [US]\nRUECKES THOMAS [US]\nW...,"\nNANTERO INC [US]\nNANTERO, INC\n",EP2070088\n,\nEP20070840800\n,2007-08-08\n,\nWO2007US75521\n2007-08-08\nUS20060836343P\n2...,\nG11C11/00\nG11C11/14\nG11C11/56\nG11C13/02\n...,\nB82Y10/00\nG11C11/56\nG11C13/00R25V\nG11C13/...,"Under one aspect, a non-volatile nanotube diod...",\nWhat is claimed is:\n1. A non-volatile nanot...,EP2070088A4;WO2008021900A2;WO2008021900A3;WO20...,"NONVOLATILE RESISTIVE MEMORIES, LATCH CIRCUITS..."


##### Reason I picked doc2vec model is my familiarity with Gensim. I have worked a lot with Gensim for Latent dirichlet allocation topic modelling, so I am much more comfortable and doc2vec is very easy to play with and get results in short time

In [ ]:
#END